# Coordinate Units and Gradient Calculations

This notebook demonstrates how coordinate units work when computing gradients and derivatives. It builds on the units system introduced in Notebook 12 and the Poisson validation examples from Notebook 5a.

**Key concepts:**
- Meshes can have coordinate units (e.g., meters, kilometers)
- Gradients automatically scale with coordinate units
- Physical dimensions are preserved through calculations

In [1]:
# Fix for visualization in interactive notebooks
import nest_asyncio
nest_asyncio.apply()

import underworld3 as uw
import numpy as np
import sympy

## Problem Setup: Linear Temperature Profile

We solve a simple heat diffusion problem with known analytical solution:

$$\nabla^2 T = 0$$

with boundary conditions:
- $T(y=0) = 300$ K (bottom)
- $T(y=L_y) = 1600$ K (top)

The analytical solution is: $T(y) = T_0 + \frac{\Delta T}{L_y} y$

This gives a constant gradient: $\frac{\partial T}{\partial y} = \frac{\Delta T}{L_y}$

We'll solve this problem on two different meshes:
1. Mesh with coordinates in **meters**
2. Mesh with coordinates in **kilometers**

The gradient values should differ by a factor of 1000 (K/m vs K/km).

## Case 1: Mesh in Meters

First, create a mesh with coordinates specified in meters.

In [2]:
# Physical domain: 1000m × 500m
L_x_m = 1000.0  # meters
L_y_m = 500.0   # meters

# Temperature boundary conditions
T_bottom = 300.0   # K
T_top = 1600.0     # K
Delta_T = T_top - T_bottom  # 1300 K

# Create mesh with meter units
mesh_m = uw.meshing.StructuredQuadBox(
    elementRes=(16, 16),
    minCoords=(0.0, 0.0),
    maxCoords=(L_x_m, L_y_m),
    units="meter"  # Coordinate units
)

print(f"Mesh in meters:")
print(f"  Domain: {L_x_m} m × {L_y_m} m")
print(f"  Coordinate units: {mesh_m.units}")
print(f"  Expected gradient: ΔT/Ly = {Delta_T/L_y_m:.3f} K/m")

Structured box element resolution 16 16
Mesh in meters:
  Domain: 1000.0 m × 500.0 m
  Coordinate units: meter
  Expected gradient: ΔT/Ly = 2.600 K/m


### Solve for Temperature

Set up and solve the Poisson equation.

In [3]:
# Create temperature variable
T_m = uw.discretisation.MeshVariable(
    "T_m", mesh_m, 1, degree=2, units="kelvin"
)

# Set up Poisson solver
poisson_m = uw.systems.Poisson(mesh_m, u_Field=T_m)
poisson_m.constitutive_model = uw.constitutive_models.DiffusionModel
poisson_m.constitutive_model.Parameters.diffusivity = 1
poisson_m.f = 0.0  # No source term

# Apply boundary conditions
poisson_m.add_dirichlet_bc(T_bottom, "Bottom")
poisson_m.add_dirichlet_bc(T_top, "Top")

# Solve
poisson_m.solve()
print(f"Solver converged: {poisson_m.snes.getConvergedReason() > 0}")

Solver converged: True


### Compute Gradient

Use a projection solver to recover the gradient field from the temperature solution.

In [4]:
# Create gradient variable (vector field)
gradT_m = uw.discretisation.MeshVariable(
    "gradT_m", mesh_m, mesh_m.dim, degree=1
)

# Set up gradient projection
x_m, y_m = mesh_m.X
gradient_proj_m = uw.systems.Vector_Projection(mesh_m, gradT_m)
gradient_proj_m.uw_function = mesh_m.vector.gradient(T_m.sym)
gradient_proj_m.solve()

# Evaluate gradient at center of domain
x_center_m = L_x_m / 2
y_center_m = L_y_m / 2
grad_m = uw.function.evaluate(
    gradT_m.sym, np.array([[x_center_m, y_center_m]])
)

dT_dx_m = grad_m[0, 0, 0]  # Should be ~0 (no x-variation)
dT_dy_m = grad_m[0, 0, 1]  # Should be ΔT/Ly

print(f"Gradient with meter coordinates:")
print(f"  ∂T/∂x = {dT_dx_m:.6f} K/m (expected: 0)")
print(f"  ∂T/∂y = {dT_dy_m:.6f} K/m (expected: {Delta_T/L_y_m:.3f})")

ValueError: Cannot add array with units 'meter' and dimensionless array. Convert to same unit system first.

## Case 2: Mesh in Kilometers

Now solve the same physical problem, but with the mesh specified in kilometers.

In [ ]:
# Same physical domain, different units
L_x_km = L_x_m / 1000.0  # 1.0 km
L_y_km = L_y_m / 1000.0  # 0.5 km

# Create mesh with kilometer units
mesh_km = uw.meshing.StructuredQuadBox(
    elementRes=(16, 16),
    minCoords=(0.0, 0.0),
    maxCoords=(L_x_km, L_y_km),
    units="kilometer"  # Coordinate units
)

print(f"Mesh in kilometers:")
print(f"  Domain: {L_x_km} km × {L_y_km} km")
print(f"  Coordinate units: {mesh_km.units}")
print(f"  Expected gradient: ΔT/Ly = {Delta_T/L_y_km:.3f} K/km")

### Solve and Compute Gradient

In [ ]:
# Temperature variable
T_km = uw.discretisation.MeshVariable(
    "T_km", mesh_km, 1, degree=2, units="kelvin"
)

# Poisson solver
poisson_km = uw.systems.Poisson(mesh_km, u_Field=T_km)
poisson_km.constitutive_model = uw.constitutive_models.DiffusionModel
poisson_km.constitutive_model.Parameters.diffusivity = 1
poisson_km.f = 0.0

# Boundary conditions (same physical temperatures)
poisson_km.add_dirichlet_bc(T_bottom, "Bottom")
poisson_km.add_dirichlet_bc(T_top, "Top")

# Solve
poisson_km.solve()

# Gradient variable
gradT_km = uw.discretisation.MeshVariable(
    "gradT_km", mesh_km, mesh_km.dim, degree=1
)

# Compute gradient
x_km, y_km = mesh_km.X
gradient_proj_km = uw.systems.Vector_Projection(mesh_km, gradT_km)
gradient_proj_km.uw_function = mesh_km.vector.gradient(T_km.sym)
gradient_proj_km.solve()

# Evaluate at center
x_center_km = L_x_km / 2
y_center_km = L_y_km / 2
grad_km = uw.function.evaluate(
    gradT_km.sym, np.array([[x_center_km, y_center_km]])
)

dT_dx_km = grad_km[0, 0, 0]
dT_dy_km = grad_km[0, 0, 1]

print(f"Gradient with kilometer coordinates:")
print(f"  ∂T/∂x = {dT_dx_km:.6f} K/km (expected: 0)")
print(f"  ∂T/∂y = {dT_dy_km:.3f} K/km (expected: {Delta_T/L_y_km:.3f})")

## Comparison: Gradient Scaling

The key result: gradients scale correctly with coordinate units.

In [ ]:
import matplotlib.pyplot as plt

# Compute scaling ratio
scaling_ratio = dT_dy_km / dT_dy_m
expected_ratio = 1000.0  # m to km conversion

print("=" * 60)
print("GRADIENT SCALING VERIFICATION")
print("=" * 60)
print(f"Meter mesh:      ∂T/∂y = {dT_dy_m:.6f} K/m")
print(f"Kilometer mesh:  ∂T/∂y = {dT_dy_km:.3f} K/km")
print(f"")
print(f"Scaling ratio:   {scaling_ratio:.2f}")
print(f"Expected ratio:  {expected_ratio:.2f}")
print(f"Relative error:  {abs(scaling_ratio - expected_ratio) / expected_ratio * 100:.2f}%")
print("=" * 60)

# Visual comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Meter mesh gradient
axes[0].arrow(0.5, 0.3, 0, 0.4, head_width=0.05, head_length=0.05, 
              fc='blue', ec='blue', linewidth=2)
axes[0].text(0.55, 0.5, f'{dT_dy_m:.3f} K/m', fontsize=12)
axes[0].set_xlim(0, 1)
axes[0].set_ylim(0, 1)
axes[0].set_title('Gradient with Meter Coordinates')
axes[0].set_xlabel('Normalized Position')
axes[0].grid(True, alpha=0.3)

# Kilometer mesh gradient
axes[1].arrow(0.5, 0.3, 0, 0.4, head_width=0.05, head_length=0.05, 
              fc='red', ec='red', linewidth=2)
axes[1].text(0.55, 0.5, f'{dT_dy_km:.0f} K/km', fontsize=12)
axes[1].set_xlim(0, 1)
axes[1].set_ylim(0, 1)
axes[1].set_title('Gradient with Kilometer Coordinates')
axes[1].set_xlabel('Normalized Position')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("media/coordinate_units_gradient_comparison.png", dpi=150, bbox_inches='tight')
plt.show()

## Physical Interpretation

The gradient represents the rate of temperature change per unit distance:

- **Meter coordinates**: $\frac{\partial T}{\partial y} = 2.6$ K/m means temperature increases by 2.6 K for each meter of depth
- **Kilometer coordinates**: $\frac{\partial T}{\partial y} = 2600$ K/km means temperature increases by 2600 K for each kilometer of depth

Both describe the same physical gradient, just in different units. The numerical values differ by exactly the unit conversion factor (1000 m/km).

## Coordinate Units and `uw.get_units()`

Coordinate units can be queried using the universal `uw.get_units()` function.

In [ ]:
print("Coordinate unit queries:")
print(f"  mesh_m.units:  {mesh_m.units}")
print(f"  mesh_km.units: {mesh_km.units}")
print(f"")
print(f"  uw.get_units(mesh_m.points):  {uw.get_units(mesh_m.points)}")
print(f"  uw.get_units(mesh_km.points): {uw.get_units(mesh_km.points)}")
print(f"")
print("Temperature variable units (independent of coordinates):")
print(f"  T_m.units:  {T_m.units}")
print(f"  T_km.units: {T_km.units}")

## Summary

This notebook demonstrated:

1. **Coordinate Units**: Meshes can be created with physical coordinate units (meters, kilometers, etc.)
2. **Gradient Scaling**: Computed gradients automatically scale with coordinate units
3. **Dimensional Consistency**: The same physical gradient has different numerical values depending on coordinate units, related by the appropriate conversion factor
4. **Unit Queries**: The `uw.get_units()` function works universally on meshes, coordinates, and variables

**Key Result**: Gradient in K/km = 1000 × Gradient in K/m (for the same physical field)

This behavior ensures dimensional consistency when working with physical units in geophysical models.

## Exercise 13.1

Modify the example to use a non-trivial source term. For instance, solve:

$$\nabla^2 T = -Q_0$$

where $Q_0$ is a constant heat production rate. What are the units of $Q_0$? How does the gradient change?

## Exercise 13.2

Create a mesh with centimeter units and solve the same problem. Verify that the gradient scaling continues to work correctly. What is the gradient in K/cm?

## Exercise 13.3

The sinusoidal profile from Notebook 5a has an analytical gradient:

$$T(y) = \sin(\pi y) \quad \Rightarrow \quad \frac{\partial T}{\partial y} = \pi \cos(\pi y)$$

Solve this problem on meshes with different coordinate units and verify that the gradient magnitudes scale correctly while the mathematical form remains the same.